In [145]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pylatex import Document, Section, Subsection,Subsubsection
from pylatex import Tabular, Table, MultiColumn
from pylatex.utils import NoEscape, bold
from pylatex.package import Package
from pylatex.base_classes import Environment
from tkinter.filedialog import askopenfilename
import shutil
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [146]:
import pint
ureg = pint.UnitRegistry()
N = ureg('N')
m = ureg('m')
mm = ureg('mm')
cm = ureg('cm')
Pa = ureg('Pa')
MPa = ureg('MPa')
pulg = ureg('in')
kN = ureg('kN')
kgf = ureg('kgf')
ureg.define('tonf=1000*kgf')
tonf = ureg('tonf')

In [147]:
#Datos de la columna:
b_col=0.7*m
h_col=0.3*m
db_col = 5/8*pulg


#Datos del suelo
q_adm=3.0*kgf/cm**2 #presion admisible del suelo.
γ_s = 1.8*tonf/m**3


#Datos de la zapata
h=0.6*m    #altura predefinida de la zapata.
Df=1.8*m   #profundidad de desplante de la zapata. (desde el NTN al NFC).

#Sobrecarga del piso
h_p = 10*cm         #altura de piso terminado
SC_p = 500*kgf/m**2 #sobrecarga del piso

#Materiales
fc = 210*kgf/cm**2
fy = 4200*kgf/cm**2
γ_c = 2.4*tonf/m**3



In [148]:
db_col.to('cm')

1.5875 <Unit('centimeter')>

In [149]:
var_dict = dict(
    var_bcol = (b_col,'m','m'),
    var_hcol = (h_col,'m','m'),
    var_dbcol = (db_col,'in','"'),
    var_dbc_cm = (db_col, 'cm','cm'),
    var_Df = (Df,'m','m'),
    var_fc = (fc,'kgf/cm**2',r'kgf/cm^2'),
    var_fy = (fy,'kgf/cm**2',r'kgf/cm^2'),
    var_gamma_s = (γ_s,'tonf/m**3',r'tonf/m^3'),
    var_gamma_c = (γ_c,'tonf/m**3',r'tonf/m^3'),
    var_SCp = (SC_p,'kgf/m**2',r'tonf/m^3'),
    var_hp = (h_p,'m','m'),
    var_sigma_s = (b_col,'m','m')
)

In [162]:
## CÁLCULOS
# Longitud de desarrollo
ld1 = 0.08*db_col.to('cm')*fy/((fc/(kgf/cm**2))**0.5*(kgf/cm**2))
ld2 = 0.004*db_col.to('cm')*fy/(kgf/cm**2)
d = max(ld1,ld2)
hz = d+0.1*m

# Presión de servicio
P_servicio = 11000*kgf
q_n = 1.5*kgf/cm**2

# area tentativa de la zapata
A_tentativa = P_servicio/q_n
A_tentativa

# Dimensiones de la zapata
Long_zap_L = np.sqrt(A_tentativa) + (b_col + h_col) / 2
Long_zap_B = np.sqrt(A_tentativa) - (b_col + h_col) / 2
Area_total = Long_zap_L * Long_zap_B # área

# Presiones en las esquinas de la zapata
sigma1 = 1*kgf/cm**2
sigma2 = 0.9*kgf/cm**2
sigma3 = 1.5*kgf/cm**2
sigma4 = 1.2*kgf/cm**2

sigma_ultima = sigma3
# Verificación de corte por punzonamiento
secc_criticaP = 2*(b_col + d) + 2*(h_col + d)   # Sección crítica
area_tributaP = (b_col + d) * (h_col + d)       # Área tributaria

    # Cortante de diseño
vu_punz = sigma_ultima*(Area_total - area_tributaP)

    # Cortante último por punzonamiento
vc_ultimP1 = 10*kgf/cm**2
vc_ultimP2 = 9*kgf/cm**2
vc_ultimP3 = 7*kgf/cm**2



In [164]:
vu_punz.to('kN')

-33.866957260715104 <Unit('kilonewton')>

In [152]:
aa = P_servicio.to('N')
bb = q_n.to('N/mm**2')
cc = aa/bb
cc.to('cm**2')
cc

733333.3333333333 <Unit('millimeter ** 2')>

In [165]:
# Nuevo  diccionario
var_dict.update(dict(
var_ld1 = (ld1,'cm','cm'),
var_ld2 = (ld2,'cm','cm'),
var_ldes = (d,'cm','cm'),
var_hz = (hz,'cm','cm'),
var_P_servicio = (P_servicio,'N', 'N'),
var_q_n = (q_n,'N/mm**2',r'N/mm^2'),
var_A_tentativa = (A_tentativa,'mm**2',r'mm^2'),
var_Long_zap_L = (Long_zap_L,'m','m'),
var_Long_zap_B = (Long_zap_B,'m','m'),
var_Area_total = (Area_total,'m**2', r'm^2'),
var_sigma1 = (sigma1, 'MPa', 'MPa'),
var_sigma2 = (sigma2, 'MPa', 'MPa'),
var_sigma3 = (sigma3, 'MPa', 'MPa'),
var_sigma4 = (sigma4, 'MPa', 'MPa'),
var_sigmaUltima = (sigma_ultima,'MPa', 'MPa'),
var_secc_criticaP = (secc_criticaP, 'm', 'm'),
var_area_tributaP = (area_tributaP, 'm**2', r'm^2'),
var_vu_punz = (vu_punz,'kN','kN'),
vc_ultimP1 = (vc_ultimP1, 'MPa', MPa),
vc_ultimP = (vc_ultimP2, 'MPa', MPa),
vc_ultimP1 = (vc_ultimP3, 'MPa', MPa),
))

In [166]:
# Definir la cantidad en mm²
cantidad_en_mm2 = 1000 * ureg('mm^2')

# Convertir la cantidad a cm²
cantidad_en_cm2 = cantidad_en_mm2.to('cm^2')

# Imprimir la cantidad en cm²
print(cantidad_en_cm2)

10.0 centimeter ** 2


In [167]:
import zapata_utils as zp
cuerpo = zp.cuerpo
cuerpo = zp.set_variablesUnit(var_dict,cuerpo)

In [168]:
geometry_options = { "left": "2.5cm", "top": "1.5cm" }
doc = Document(geometry_options=geometry_options)
doc.packages.append(NoEscape(zp.preambulo))

7

In [169]:
doc.append(NoEscape(cuerpo))

In [170]:
#doc.generate_tex('out2/memoriaZapata')
# doc.generate_pdf('out2/memoriaZapata')